### Create CSV from midi sources

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import json
import music21
import glob
import os
from tqdm import tqdm
from IPython.display import Image, Audio
import traceback

In [3]:
# parallel
import concurrent
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from fastprogress.fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import MasterBar, ProgressBar

In [4]:
from midi_data import *

In [5]:
path = Path('../data/midi')

In [6]:
directories = [x for x in path.iterdir() if x.is_dir()]; directories

[PosixPath('../data/midi/cprato_midi'),
 PosixPath('../data/midi/Wikifonia'),
 PosixPath('../data/midi/transposed'),
 PosixPath('../data/midi/hymn'),
 PosixPath('../data/midi/classic_piano_scrape'),
 PosixPath('../data/midi/touhou_scrape'),
 PosixPath('../data/midi/hooktheory'),
 PosixPath('../data/midi/transcribedc_v1'),
 PosixPath('../data/midi/ecomp_midi'),
 PosixPath('../data/midi/freemidi_scrape'),
 PosixPath('../data/midi/piano_e_competition'),
 PosixPath('../data/midi/midiworld_scrape')]

## General stream parsing

In [7]:
def get_music21_attr(fp, transpose=True, offset=None):
    try:
        stream = music21.converter.parse(fp)
    except Exception as e:
        print('Could not parse stream', fp, e)
#         traceback.print_exc()
        return {}
    stream_attr = get_stream_attr(stream)
    if transpose:
        transposed_file = transpose_midi2c(fp, stream, halfsteps=offset)
        transposed_stream = music21.converter.parse(fp)
        t_key = transposed_stream.analyze('key')
        transposed_attr = {
            'inferred_keyc': f'{t_key.tonic.name} {t_key.mode}',
            'midi_keyc': str(transposed_file),
        }
        stream_attr = {**stream_attr, **transposed_attr}
    return stream_attr 

In [8]:
def get_stream_attr(s):
    instruments = [i.instrumentName for i in list(s.getInstruments(recurse=True)) if i.instrumentName]
    metronome = list(filter(lambda x: isinstance(x, music21.tempo.MetronomeMark), s.flat))[0]
    bpm = metronome.getQuarterBPM()
    key = s.analyze('key')
    s_flat = s.flat
    time_sig = s_flat.timeSignature.ratioString if hasattr(s_flat.timeSignature, 'ratioString') else None
    return {
        'instruments': instruments,
        'bpm': bpm,
        'inferred_key': f'{key.tonic.name} {key.mode}',
        'seconds': s_flat.seconds,
        'time_signature': time_sig,
    }

In [40]:
def process_parallel(func, arr, key_func=None, max_workers=None):
    "Maps function to an array of objects. Key can me modified by passing a key_func"
    results = {}
    with ProcessPoolExecutor(max_workers=max_workers) as ex:
        futures = {ex.submit(func,o):o for i,o in enumerate(arr)}
        for f in progress_bar(concurrent.futures.as_completed(futures), total=len(arr)):
            key = futures[f]
            if key_func: key = key_func(key)
            results[key] = f.result()
    return results

In [32]:
import csv
import pandas as pd

def format_values(d):
    def format_value(v):
        if isinstance(v, list): return ','.join(v)
        return v
    return {k:format_value(v) for k,v in d.items()}

def arr2csv(arr, out_file):
    all_keys = {k for d in arr for k in d.keys()}
    arr = [format_values(x) for x in arr]
    with open(out_file, 'w') as f:
        dict_writer = csv.DictWriter(f, list(all_keys))
        dict_writer.writeheader()
        dict_writer.writerows(arr)

### Gather Hooktheory Data

In [11]:
hook_path = path/'hooktheory'
hook_info_path = hook_path/'key2info.json'
hook_metapath = hook_path/'song_metadata.json'

def song_key(s): return '_'.join(s.parts[-3:-1])

In [12]:
key2info = load_json(hook_info_path)

if key2info is None:
    song_info = list((hook_path/'xml').glob('*/*/*/*.json'))
    key2info = {song_key(s):json.load(open(s, 'r')) for s in song_info}
    save_json(key2info, hook_info_path)
    len(song_info)

In [13]:
# song_xml = list((hook_path/'xml').glob('*/*/*/*.xml')); len(song_xml)
song_json = list((hook_path/'event').glob('*/*/*/*_key.json')); len(song_json)
# song_mid = list((hook_path/'pianoroll').glob('*/*/*/*_nokey.mid')); len(song_mid)

19876

In [13]:
song_json[0]

PosixPath('../data/midi/hooktheory/event/w/wayne-sharpe/yu-gi-oh-theme-song/chorus_symbol_key.json')

In [14]:
def get_hooktheory_attr(args):
    song_json_path,song_info = args
    song_json = json.load(open(song_json_path, 'r'))
    metadata = song_json['metadata']
    artist = song_json_path.parts[-3]
    title = song_json_path.parts[-2]
    section = song_json_path.name.split('_')[0]
    midi_path = str(song_json_path.with_suffix('.mid')).replace('event', 'pianoroll').replace('symbol_', '')
    
    # convert stream here
    metadata = {
        'artist': artist,
        'section': section,
        'original_path': midi_path,
        'parts': song_info['section'],
        'song_url': song_info['song_url'],
        'genres': song_info['genres'],
        'midi_title': metadata['title'],
        'source': 'hooktheory',
        'ht_bpm': metadata['BPM'],
        'ht_mode': metadata['mode'],
        'ht_key': metadata['key'],
        'ht_time_signature': metadata['beats_in_measure']
    }
    mode = metadata['ht_mode']
    if mode is None:
        print('No mode found. Assuming major', song_json_path)
        mode = 'major'
    offset = keyc_offset(metadata['ht_key'], mode)
    stream_info = get_music21_attr(midi_path, transpose=True, offset=offset)
    return {**metadata, **stream_info}

In [14]:
# sanity check
# hook_out = get_hooktheory_attr(song_json[1000], key2info); hook_out

In [15]:
def ht_key(fp): return ''.join(fp.with_suffix('').parts[-3:])

In [17]:
file2metadata = load_json(hook_metapath)
if file2metadata is None: file2metadata = {}

In [18]:
subset = song_json
new_files = [(fp,key2info[song_key(fp)]) for fp in subset if ht_key(fp) not in file2metadata] # pass file and song info

In [19]:
processed = process_parallel(get_hooktheory_attr, new_files, key_func=lambda x: ht_key(x[0]))
file2metadata.update(processed); len(file2metadata)

No mode found. Assuming major ../data/midi/hooktheory/event/w/willie-nelson/blue-eyes-cryin/chorus_symbol_key.json


No mode found. Assuming major ../data/midi/hooktheory/event/w/weezer/beverly-hills/intro-and-verse_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/w/weezer/beverly-hills/intro-and-verse_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-25-in-g-minor/intro-and-verse_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/w/woody-guthrie/this-land-is-your-land/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/w/ween/roses-are-free/chorus_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/y/ylvis/the-fox---what-does-the-fox-say/verse-and-pre-chorus_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/j/junior-boys/first-

No mode found. Assuming major ../data/midi/hooktheory/event/b/blast/malaguena/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/b/benjamin-ingrosso/good-lovin/chorus_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/b/benjamin-ingrosso/good-lovin/chorus_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/b/belle-and-sebastian/like-dylan-in-the-movies/chorus_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/b/broken-social-scene/anthems-for-a-spianorolleen-year-old-girl/verse_key.mid File not found or no such format found for: ../data/midi/hooktheory/pianoroll/b/broken-social-scene/anthems-for-a-spianorolleen-year-old-girl/verse_key.mid
No mode found. Assuming major ../data/midi/hooktheory/event/b/b-o-b-ft-hayley-williams/airplanes/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/b/bill-withers/lean-on-me/intro_symbol_key.json
No mode f

No mode found. Assuming major ../data/midi/hooktheory/event/c/counting-crows/a-long-december/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/c/christina-perri/jar-of-hearts/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/e/eric-clapton/you-look-wonderful-tonight/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/e/eric-clapton/you-look-wonderful-tonight/intro_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/e/eminem-ft-rihanna/love-the-way-you-lie/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/e/eminem-ft-rihanna/love-the-way-you-lie/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/e/equinox/bones/intro_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/e/equinox/bones/intro_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/e/elto

No mode found. Assuming major ../data/midi/hooktheory/event/t/train/hey-soul-sister/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/t/the-rolling-stones/beast-of-burden/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/t/the-rolling-stones/time-is-on-my-side/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/t/taylor-swift/starlight/solo_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/t/taylor-swift/you-belong-with-me/intro_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/t/taichi-mukai/reset-%2528kaze-ga-tsuyoku-fuiteiru%2529/intro_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/t/taichi-mukai/reset-%2528kaze-ga-tsuyoku-fuiteiru%2529/intro_key.mid list index out of range
No mode found. Assuming major ../data/midi/hooktheory/event/t/taichi-mukai/reset/_symbol_key.json
Could not parse stream ../data

No mode found. Assuming major ../data/midi/hooktheory/event/o/one-republic/say/verse-and-pre-chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/one-republic/secrets/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/of-monsters-and-men/little-talks/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/oasis-and-microsoft-songsmith/wonderwall---songsmith-techno-mix/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/one-direction/little-things/verse_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/one-direction/what-makes-you-beautiful/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/o/one-direction/what-makes-you-beautiful/verse_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/k/katy-pary/fireworks/chorus_key.mid list index out of range
Could not parse stream ../data/midi

No mode found. Assuming major ../data/midi/hooktheory/event/m/miss-a/good-bye-baby/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/maroon-5/never-gonna-leave-this-bed/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/matt-nathanson/come-on-get-higher/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/matt-nathanson/come-on-get-higher/verse-and-pre-chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/michael-jackson/you-are-not-alone/bridge_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/michael-jackson/black-or-white/chorus_symbol_key.json
No mode found. Assuming major ../data/midi/hooktheory/event/m/michael-jackson/black-or-white/verse_symbol_key.json
Could not parse stream ../data/midi/hooktheory/pianoroll/q/queen/bohemian-rhapsody/outro_key.mid list index out of range
Could not parse stream ../data/midi/hooktheory/pia

In [20]:
save_json(file2metadata, hook_metapath)

Save song_list

In [21]:
ht_csv_path = 'hooktheory_metadata.csv'
file2metadata = load_json(hook_metapath)

In [33]:
arr2csv(file2metadata.values(), ht_csv_path)

In [34]:
df = pd.read_csv(ht_csv_path); df.head()

,seconds,original_path,bpm,inferred_key,ht_time_signature,song_url,inferred_keyc,time_signature,section,ht_bpm,ht_key,source,artist,parts,genres,ht_mode,midi_title,midi_keyc,instruments
0,15.000000,../data/midi/hooktheory/pianoroll/w/wayne-shar...,128.0,C major,4,https://www.hooktheory.com/theorytab/view/wayn...,C major,4/4,chorus,128,C,hooktheory,wayne-sharpe,"intro,chorus",NaN,1.0,yu-gi-oh3,../data/midi/transposed/hooktheory/pianoroll/w...,"Piano,Piano"
1,25.411765,../data/midi/hooktheory/pianoroll/w/wayne-shar...,85.0,C major,3,https://www.hooktheory.com/theorytab/view/wayn...,C major,3/4,intro,85,C,hooktheory,wayne-sharpe,"intro,chorus",NaN,1.0,yu-gi-oh,../data/midi/transposed/hooktheory/pianoroll/w...,Piano
2,17.777778,../data/midi/hooktheory/pianoroll/w/wham/last-...,108.0,B- minor,4,https://www.hooktheory.com/theorytab/view/wham...,B- minor,4/4,chorus,108,Db,hooktheory,wham,"intro,verse,chorus",Holiday,1.0,Last Christmas Chorus,../data/midi/transposed/hooktheory/pianoroll/w...,"Piano,Piano"
3,12.631579,../data/midi/hooktheory/pianoroll/w/whiteflame...,152.0,D minor,4,https://www.hooktheory.com/theorytab/view/whit...,D minor,4/4,pre-chorus,152,D,hooktheory,whiteflame,"verse,pre-chorus,chorus","J-Pop,Pop",6.0,senbonzakura - pre-Pre-Chorus,../data/midi/transposed/hooktheory/pianoroll/w...,"Piano,Piano"
4,35.555556,../data/midi/hooktheory/pianoroll/w/wham/last-...,108.0,B- minor,4,https://www.hooktheory.com/theorytab/view/wham...,B- minor,4/4,verse,108,Db,hooktheory,wham,"intro,verse,chorus",Holiday,1.0,Last Christmas Verse,../data/midi/transposed/hooktheory/pianoroll/w...,"Piano,Piano"


### Gather FreeMidi

In [35]:
freemidi_path = path/'freemidi_scrape'; list(freemidi_path.glob('*'))

[PosixPath('../data/midi/freemidi_scrape/genre-disco'),
 PosixPath('../data/midi/freemidi_scrape/genre-pop'),
 PosixPath('../data/midi/freemidi_scrape/genre-dance-eletric'),
 PosixPath('../data/midi/freemidi_scrape/genre-punk'),
 PosixPath('../data/midi/freemidi_scrape/genre-hip-hop-rap'),
 PosixPath('../data/midi/freemidi_scrape/genre-rock'),
 PosixPath('../data/midi/freemidi_scrape/freemidi_dance_metadata.json'),
 PosixPath('../data/midi/freemidi_scrape/freemidi_pop_metadata.json')]

In [36]:
def parse_freemidi_songs(fp, genre=None):
    metadata,attr = {},{}
    try:
        name = fp.with_suffix('').name.split(' - ')
        artist = name[0]
        title = name[-1]
        metadata = {
            'artist': artist,
            'title': title,
            'midi': str(fp),
            'genre': genre,
            'source': 'freemidi'
        }
    except Exception as e:
        print('Exeption:', fp, e)
    try:
        attr = get_music21_attr(fp)
    except Exception as e:
        print('Exeption:', fp, e)
    return {**metadata, **attr}

In [47]:
def parse_freemidi_genre(genre_path, genre, limit=None):
    metadata_path = genre_path.parent/f'freemidi_{genre}_metadata.json'
    file2metadata = load_json(metadata_path)
    if file2metadata is None: file2metadata = {}
        
    file_list = list(genre_path.glob('*'))
    if limit: file_list = file_list[:limit]
        
    parsed = process_parallel(partial(parse_freemidi_songs, genre=genre), file_list, key_func=str)
    file2metadata.update(parsed)
#     for fp in tqdm(file_list, total=len(file_list)):
#         file2metadata[fp.with_suffix('').name] = parse_freemidi_songs(fp, genre)
    
    json.dump(file2metadata, open(metadata_path, 'w'))
    
    return file2metadata, metadata_path

In [48]:
# sanity check
# dance_files = (freemidi_path/'genre-dance-eletric/').ls()
# freemidi_out = parse_freemidi_songs(dance_files[0], genre='dance'); freemidi_out

In [50]:
freemidi_dance_list, freemidi_dance_path = parse_freemidi_genre(freemidi_path/'genre-dance-eletric', 'dance')

Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - Not The Red Baron.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Freelove.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Bjork - Glora.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - I'm Not Givin' You Up.mid badly formated midi bytes, got: b'RIFFB\x8c\x00\x00RMIDdata~\x8b\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Atomic Kitten - Whole Again.mid badly formated midi bytes, got: b'RIFF\x08K\x00\x00RMIDdata{J\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Apollo 440 - 

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=8, channel=None>: unknown midi event type: None; ignored.


Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Ya Tvoy Vrag (I'm Your Enemy).mid badly formated midi bytes, got: b'RIFF,\xa3\x00\x00RMIDdata\xc6\xa2\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Get On Your Feet.mid badly formated midi bytes, got: b'RIFF\x92\xa8\x01\x00RMIDdata\xd8\xa7\x01\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Mi Tierra.mid badly formated midi bytes, got: b'RIFFz\r\x01\x00RMIDdata\xce\x0c\x01\x00'
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - 10000 Oceans.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Breathe.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/New Order - T

midi: WARNING: Conversion error for <MidiEvent None, t=None, track=0, channel=None>: unknown midi event type: None; ignored.


Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Whitney Houston - If I Told You That.mid badly formated midi bytes, got: b'error with file'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Coming Out of The Dark.mid badly formated midi bytes, got: b'RIFF\xca)\x00\x00RMIDdata\x03)\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Enigma - Sadness.mid badly formed midi string: missing leading MTrk
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Tori Amos - Here In My Head.mid list index out of range
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Ne Ver Ne Boysia.mid badly formated midi bytes, got: b'RIFF\xa8\xbd\x00\x00RMIDdata\x13\xbd\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Mitierra.mid badly formated midi bytes, got: b'RIFFz\r\x01\x00RMIDdata\xce\x0c\x01\x00'
Could not parse stream ../data/mid

...

Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - I Feel Loved.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Modern Talking - Megamix 2000.mid badly formated midi bytes, got: b'RIFFp\x91\x02\x00RMIDdata\xd1\x90\x02\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Aqua - Turn Back Time.mid index out of range
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Moby - Natural Blues.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Donna Summer - Bad Girls.mid badly formed midi string: missing leading MTrk
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Abriendo Puertas.mid badly formated midi bytes, got: b'error with file'
Could not parse stre

midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-3, velocity=2>: Problem with representing either -3 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-3, velocity=2>: Problem with representing either -3 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NO

Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Gloria Estefan - Cant Stay Away From You.mid badly formated midi bytes, got: b'RIFF^5\x00\x00RMIDdata\x934\x00\x00'
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Depeche Mode - Fools.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-1, velocity=2>: Problem with representing either -1 or 2; ignored.


Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/New Order - Ceremony.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data/midi/freemidi_scrape/genre-dance-eletric/Tatu - Robot.mid badly formated midi bytes, got: b'RIFF\x82N\x00\x00RMIDdata\x1cN\x00\x00'
Exeption: ../data/midi/freemidi_scrape/genre-dance-eletric/Calvin Harris - Summer.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.


In [ ]:
freemidi_pop_list, freemidi_pop_path = parse_freemidi_genre(freemidi_path/'genre-pop', 'pop')

Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Cyndi Lauper - Whats Going On.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Cyndi Lauper - Who Let In The Rain.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'
Exeption: ../data/midi/freemidi_scrape/genre-pop/Chris De Burgh - Discovery.mid list index out of range


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_OFF, t=None, track=1, channel=1, pitch=-2, velocity=2>: Problem with representing either -2 or 2; ignored.


Exeption: ../data/midi/freemidi_scrape/genre-pop/Sting - A Thousand Years.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-pop/Michael Jackson - BilliJeans_AI_composed.mid list index out of range
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/5th Dimension - One Less Bell To Answere.mid badly formed midi string: missing leading MTrk
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Cyndi Lauper - The World Is Stone.mid badly formated midi bytes, got: b'0&\xb2u\x8ef\xcf\x11\xa6\xd9\x00\xaa\x00b\xcel?\x13\x00\x00'


midi: WARNING: Conversion error for <MidiEvent None, t=None, track=10, channel=None>: unknown midi event type: None; ignored.


Exeption: ../data/midi/freemidi_scrape/genre-pop/Elton John - Blue Eyes.mid list index out of range


...

Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Floyd Cramer - Last date.mid badly formated midi bytes, got: b'error with file'


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=2, channel=2, pitch=-2, velocity=51>: Problem with representing either -2 or 51; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=2, channel=2, pitch=-2, velocity=0>: Problem with representing either -2 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=3, pitch=-2, velocity=67>: Problem with representing either -2 or 67; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=3, channel=3, pitch=-2, velocity=0>: Problem with representing either -2 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=6, channel=4, pitch=-2, velocity=100>: Problem with representing either -2 or 100; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=6, channel=4, pitch=-2, velocity=0>: Problem with representing either -2 or 0; ignored.


Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Gloria Estefan - Anything For You.mid badly formated midi bytes, got: b'RIFF\x00)\x00\x00RMIDdataE(\x00\x00'


midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=5, channel=4, pitch=-5, velocity=1>: Problem with representing either -5 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=5, channel=4, pitch=-5, velocity=0>: Problem with representing either -5 or 0; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=5, channel=4, pitch=-5, velocity=1>: Problem with representing either -5 or 1; ignored.
midi: WARNING: Conversion error for <MidiEvent NOTE_ON, t=None, track=5, channel=4, pitch=-5, velocity=0>: Problem with representing either -5 or 0; ignored.


Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Gloria Estefan - Breaking Up Is Hard To Do.mid badly formated midi bytes, got: b'RIFF\x10N\x00\x00RMIDdataDM\x00\x00'
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Aaron Carter - Saturday Night.mid badly formated midi bytes, got: b'error with file'
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/Natalie Cole - I Live For Your Love.mid badly formated midi bytes, got: b'<html>\n<head>\n<title'
Could not parse stream ../data/midi/freemidi_scrape/genre-pop/ABC - The Look Of Love.mid badly formed midi string: missing leading MTrk
Exeption: ../data/midi/freemidi_scrape/genre-pop/Cole Porter - Night And Day.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Exeption: ../data/midi/freemidi_scrape/genre-pop/OMD - Locomotion.mid cannot get a seconds duration when no TempoIndication classes are found in or before this Stream.
Could not parse stream ../data

### Gather Cprato

### Gather MidiWorld

### Yamaha - piano

In [ ]:
ecomp_path = Path('../data/midi/piano_e_competition')
ecomp_scrape_path = Path('../data/midi/ecomp_midi')

### Classic Piano

In [ ]:
classic_scrape_path = Path('../data/midi/classic_piano_scrape')

### Creating CSV

In [ ]:
import csv
toCSV = [{'name':'bob','age':25,'weight':200},
         {'name':'jim','age':31,'weight':180}]
keys = toCSV[0].keys()
with open('people.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)